In [1]:
# Use Google Colab
use_colab = True
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive._mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/685_Project'

    !pip install -r "/content/drive/My Drive/685_Project/requirements.txt"

!pip install sentencepiece

from datasets import load_dataset


import sys
import os
import random
import shutil
import copy
import inspect

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

import numpy as np
import torch
import transformers
import datasets
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tqdm
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer

Mounted at /content/drive
/content/drive/My Drive/685_Project
     |████████████████████████████████| 298 kB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 18.2 MB/s 
     |████████████████████████████████| 132 kB 44.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.2 MB/s 
     |████████████████████████████████| 243 kB 46.6 MB/s 
     |████████████████████████████████| 61 kB 525 kB/s 
     |████████████████████████████████| 596 kB 32.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.6 MB/s 
     |████████████████████████████████| 895 kB 35.7 MB/s 
     |████████████████████████████████| 160 kB 48.5 MB/s 
     |████████████████████████████████| 271 kB 45.6 MB/s 
     |████████████████████████████████| 192 kB 53.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [2]:
assert torch.cuda.is_available()
dataset = load_dataset('csv', data_files={'train': 'hateval2019_en_train.csv','test': 'hateval2019_en_test.csv','valid': 'hateval2019_en_dev.csv'})
# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Using custom data configuration default-fa9deb86a6bf3771


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-fa9deb86a6bf3771/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Found device: Tesla K80, n_gpu: 1


In [3]:
def mappingFunction(index, Data):
# NOT            0 0 0 -> 0
# HSP NOT  NOT   1 0 0 -> 1
# HSP TRG  NOT   1 1 0 -> 2
# HSP NOT  ARG   1 0 1 -> 3
# HSP TRG  ARG   1 1 1 -> 4

  if Data["HS"][index] == 0 and Data["TR"][index]== 0 and Data["AG"][index] == 0:
    return 0
  elif Data["HS"][index] == 1 and Data["TR"][index]== 0 and Data["AG"][index] == 0:
    return 1
  elif Data["HS"][index] == 1 and Data["TR"][index]== 1 and Data["AG"][index] == 0:
    return 2
  elif Data["HS"][index] == 1 and Data["TR"][index]== 0 and Data["AG"][index] == 1:
    return 3
  elif Data["HS"][index] == 1 and Data["TR"][index]== 1 and Data["AG"][index] == 1:
    return 4

In [4]:
# dataset["train"]

In [5]:
# dataset["train"][0]

In [6]:
# Convert to Multi-Class
classCol = [0] * len(dataset["train"])

for index in range(len(dataset["train"])):
  classCol[index]  =  mappingFunction(index, dataset["train"])
dataset["train"] = dataset["train"].add_column("class", classCol)

In [7]:
classCol = [0] * len(dataset["valid"])

for index in range(len(dataset["valid"])):
  classCol[index]  =  mappingFunction(index, dataset["valid"])
dataset["valid"] = dataset["valid"].add_column("class", classCol)

In [8]:
classCol = [0] * len(dataset["test"])

for index in range(len(dataset["test"])):
  classCol[index]  =  mappingFunction(index, dataset["test"])
dataset["test"] = dataset["test"].add_column("class", classCol)

In [9]:
dataset["train"]

Dataset({
    features: ['id', 'text', 'HS', 'TR', 'AG', 'class'],
    num_rows: 9000
})

In [10]:
dataset["test"]

Dataset({
    features: ['id', 'text', 'HS', 'TR', 'AG', 'class'],
    num_rows: 3000
})

In [11]:
dataset["valid"]

Dataset({
    features: ['id', 'text', 'HS', 'TR', 'AG', 'class'],
    num_rows: 1000
})

In [12]:
train_dataloader = torch.utils.data.DataLoader(dataset['train'], shuffle=True, batch_size=8)
val_dataloader = torch.utils.data.DataLoader(dataset['valid'], shuffle=True, batch_size=8)
test_dataloader = torch.utils.data.DataLoader(dataset['test'], shuffle=True, batch_size=8)

In [13]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', 
    num_labels = 5, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    cache_dir='./roberta_cache'
)
model.to(device) 
batch_size = 99
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
epochs = 5

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [15]:
# os.mkdir("trained_model_hateval_baseline_multiclass")

In [16]:
import numpy as np

y_pred =[]
y_true = []
incorrect = []

# function to get validation accuracy
def get_validation_performance(val_dataloader):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    #total_eval_loss = 0

    total_correct = 0

    for batch in val_dataloader:
      if len(batch) == 0: continue
      inputs = tokenizer(batch['text'],padding='longest',return_tensors="pt").to(device)
      labels = batch['class'].to(device)
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits
            
        # Accumulate the validation loss.
        #total_eval_loss += loss.item()
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        incorrect.extend([batch["text"][index] for index in range(len(batch)) if pred_flat[index] != labels_flat[index]])
        y_pred.extend(pred_flat)
        y_true.extend(labels_flat)

        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / (len(val_dataloader) * val_dataloader.batch_size)
    return avg_val_accuracy, incorrect, y_pred, y_true


In [17]:
import random

for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    total_train_loss = 0
    model.train()
    for batch in tqdm.notebook.tqdm(train_dataloader):
      if len(batch) == 0: continue
      inputs = tokenizer(batch['text'],padding='longest',return_tensors="pt").to(device)
      labels = batch['class'].to(device)
      # Clear the previously calculated gradient
      model.zero_grad()        
      # Perform a forward pass (evaluate the model on this training batch).
      outputs = model(**inputs, labels=labels)
      loss = outputs.loss
      logits = outputs.logits
      total_train_loss += loss.item()
      # Perform a backward pass to calculate the gradients.
      loss.backward()
      # Update parameters and take a step using the computed gradient.
      optimizer.step()
    # ========================================
    #               Validation
    # ========================================
    print(f"Total loss: {total_train_loss}")
    val_acc, _, _, _ = get_validation_performance(val_dataloader)
    print(f"Validation accuracy: {val_acc}")
    torch.save(model.state_dict(),os.path.join('./trained_model_hateval_baseline_multiclass/', 'multiclass_baseline_{}.pt'.format(epoch_i)))
    
print("")
print("Training complete!")

#get_validation_performance(test_dataloader)


======== Epoch 1 / 5 ========
Training...


  0%|          | 0/1125 [00:00<?, ?it/s]

Total loss: 1313.3925559073687
Validation accuracy: 0.573

======== Epoch 2 / 5 ========
Training...


  0%|          | 0/1125 [00:00<?, ?it/s]

Total loss: 1336.3025759756565
Validation accuracy: 0.602

======== Epoch 3 / 5 ========
Training...


...

Total loss: 1333.991203069687
Validation accuracy: 0.602

======== Epoch 4 / 5 ========
Training...


  0%|          | 0/1125 [00:00<?, ?it/s]

Total loss: 1341.6995362341404
Validation accuracy: 0.603

======== Epoch 5 / 5 ========
Training...


  0%|          | 0/1125 [00:00<?, ?it/s]

Total loss: 1383.9201376736164
Validation accuracy: 0.573

Training complete!


In [18]:
y_pred = []
y_true = []
incorrect = []

In [19]:
acc, incorrect_examples, y_pred, y_true =  get_validation_performance(test_dataloader)

In [20]:
from sklearn.metrics import classification_report
report = classification_report(y_true=y_true, y_pred=y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.58      1.00      0.73      1740
           1       0.00      0.00      0.00       320
           2       0.00      0.00      0.00       346
           3       0.00      0.00      0.00       411
           4       0.00      0.00      0.00       183

    accuracy                           0.58      3000
   macro avg       0.12      0.20      0.15      3000
weighted avg       0.34      0.58      0.43      3000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
np.savetxt("incorrect_examples_hateval_multiclass.csv", 
           incorrect,
           delimiter =", ", 
           fmt ='% s')

In [22]:
from google.colab import files
files.download("incorrect_examples_hateval_multiclass.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>